In [6]:
import pandas as pd
import numpy as np
from datetime import datetime
import os   
import sys
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy.optimize import minimize
from scipy import interpolate
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay
import warnings
import statsmodels.api as sm
from scipy.optimize import fsolve
from scipy.stats import norm
from sklearn.decomposition import PCA
import warnings
from scipy.optimize import fsolve
import numpy.polynomial.polynomial as poly
warnings.filterwarnings(action='ignore')

In [13]:
industries = pd.read_csv('./industry.csv')

In [14]:
industries['industry'] = industries.industry.str.replace('CRSP', '', regex=True)
industries['industry'] = industries.industry.str.replace('Index', '', regex=True)
industries['top_tickers'] = industries['top_tickers'].str.split()
industries = industries.explode('top_tickers').reset_index(drop=True)
industries

,industry,top_tickers
0,US Consumer Discretionary,AMZN
1,US Consumer Discretionary,TSLA
2,US Consumer Discretionary,HD
3,US Consumer Discretionary,WMT
4,US Consumer Discretionary,MCD
...,...,...
105,US Utilities,AEP
106,US Utilities,VST
107,US Utilities,D
108,US Utilities,PCG


In [6]:
vols = pd.read_csv('../daily_vol_series.csv')
vols = vols[vols['SYM_ROOT'] == vols['symbol']]

In [7]:
vols.SYM_ROOT.nunique()

106

In [9]:
vols.head()

,DATE,SYM_ROOT,symbol,ivol_t,ivol_q,month,year,ivol_q_log
0,2010-01-04,A,A,7.760087e-08,3.188671e-09,1,2010,-19.563662
1,2010-01-05,A,A,4.062151e-08,1.320020e-09,1,2010,-20.445619
2,2010-01-06,A,A,2.296710e-08,1.239236e-09,1,2010,-20.508771
3,2010-01-07,A,A,2.606387e-08,1.852384e-09,1,2010,-20.106793
4,2010-01-08,A,A,2.244471e-08,2.353240e-09,1,2010,-19.867473


In [10]:
vols = vols[vols['DATE'] <= '2023-08-31']

In [11]:
vols['month'] = pd.to_datetime(vols['DATE']).dt.month
vols['year'] = pd.to_datetime(vols['DATE']).dt.year
vols['ivol_q_log'] = np.log(vols['ivol_q'])

In [12]:
def moments(df):
    final_dfs = []
    for lag in range(1, 11):
        tmp = df.groupby(['SYM_ROOT', 'year', 'month'])['ivol_q_log']\
            .apply(lambda x: (((x - x.shift(lag))**2).sum() / (x.count() - lag)) ).to_frame(f'lag{lag}_mom')
        final_dfs.append(tmp)
    final_df = pd.concat(final_dfs, axis=1)
    return final_df

In [13]:
final = moments(vols)
final.dropna(inplace=True)

In [14]:
betas = pd.DataFrame(columns=['Betas'], index=final.index)
for idx,_ in final.iterrows():
    y = final.loc[idx].values
    X = np.log(range(1, 11)).reshape(-1,1)
    # X = sm.add_constant(X)
    mod = sm.OLS(y, X).fit()
    betas.loc[idx, 'Betas'] = mod.params
betas.reset_index(inplace=True)


In [16]:
betas[betas['Betas']>=0.5]
betas['Realized_H'] = betas['Betas'] * 0.5

In [32]:
betas[]

,SYM_ROOT,year,month,Betas
0,AMC,2014,1,0.6801783537114248
1,AMC,2014,2,0.3044701508508652
2,AMC,2014,3,0.43894627795209695
3,AMC,2014,4,0.3677223030178472
4,AMC,2014,5,0.16202374674351883
...,...,...,...,...
4814,ZI,2023,3,4.1764630805925655
4815,ZI,2023,4,0.10833338739933156
4816,ZI,2023,5,0.1918529306195535
4817,ZI,2023,6,0.07078159863146065
